This notebook involves new preprocessing steps (added feature engeneering, which was implemented on 22/11 at 22:58)

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pickle
from pathlib import Path

data_path = Path("../data/")   # full path to the HEF folder

train = pd.read_csv(data_path / "mimic_train_HEF.csv", low_memory=False)
test  = pd.read_csv(data_path / "mimic_test_HEF.csv",  low_memory=False)

train.shape, test.shape


((20885, 44), (5221, 39))

In [2]:
# =============================================================================
# SAVE TEST IDs FIRST (BEFORE DROPPING!)
# =============================================================================

test_ids = test['icustay_id'].copy()
print(f"\n✓ Saved {len(test_ids)} test IDs for submission")


✓ Saved 5221 test IDs for submission


In [3]:
# =============================================================================
# 2. DROP LEAKAGE COLUMNS
# =============================================================================
columns_to_drop = [
    'DISCHTIME', 'DEATHTIME', 'DOD', 'LOS',
    'subject_id', 'hadm_id', 'icustay_id',
    'ADMITTIME', 'Diff'
]

train_clean = train.drop(columns=columns_to_drop, errors='ignore')
test_clean = test.drop(columns=columns_to_drop, errors='ignore')

In [4]:
# =============================================================================
# 3. SEPARATE TARGET
# =============================================================================
y = train_clean['HOSPITAL_EXPIRE_FLAG']
X = train_clean.drop('HOSPITAL_EXPIRE_FLAG', axis=1)
X_test = test_clean.copy()

In [5]:
# =============================================================================
# 4. IDENTIFY FEATURE TYPES
# =============================================================================
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nNumeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")


Numeric features: 24
Categorical features: 10


In [6]:
# =============================================================================
# 5. IMPUTATION
# =============================================================================
print("\n--- Imputing missing values ---")

# Numeric
numeric_imputer = SimpleImputer(strategy='median')
X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])
X_test[numeric_features] = numeric_imputer.transform(X_test[numeric_features])

# Categorical
categorical_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_features] = categorical_imputer.fit_transform(X[categorical_features])
X_test[categorical_features] = categorical_imputer.transform(X_test[categorical_features])


--- Imputing missing values ---


In [7]:
# -----------------------------------------------------------------------------
# STEP 1: Convert DOB to age (handling MIMIC-III date shifting)
# -----------------------------------------------------------------------------
print("\n--- Step 1: Converting DOB to age ---")

if 'DOB' in X.columns and 'DOB' in categorical_features:
    # MIMIC-III shifts all dates forward by ~200 years for anonymization
    # But the RELATIVE age is preserved
    # Strategy: Calculate age = ADMITTIME - DOB
    
    # Reload original data to get ADMITTIME
    train_original = pd.read_csv('../data/mimic_train_HEF.csv')
    test_original = pd.read_csv('../data/mimic_test_HEF.csv')
    
    # Convert to datetime
    dob_train = pd.to_datetime(X['DOB'], errors='coerce')
    dob_test = pd.to_datetime(X_test['DOB'], errors='coerce')
    admit_train = pd.to_datetime(train_original['ADMITTIME'], errors='coerce')
    admit_test = pd.to_datetime(test_original['ADMITTIME'], errors='coerce')
    
    # Calculate age using timedelta and convert to years
    # Use .apply() to avoid overflow
    def calculate_age(admit_time, dob):
        if pd.isna(admit_time) or pd.isna(dob):
            return np.nan
        try:
            # Calculate difference in days, then convert to years
            age_days = (admit_time - dob).days
            age_years = age_days / 365.25
            return age_years
        except:
            return np.nan
    
    # Calculate age for train
    X['age'] = [calculate_age(admit, dob) for admit, dob in zip(admit_train, dob_train)]
    
    # Calculate age for test
    X_test['age'] = [calculate_age(admit, dob) for admit, dob in zip(admit_test, dob_test)]
    
    # Convert to numeric (in case of any issues)
    X['age'] = pd.to_numeric(X['age'], errors='coerce')
    X_test['age'] = pd.to_numeric(X_test['age'], errors='coerce')
    
    # Clean up
    X = X.drop('DOB', axis=1)
    X_test = X_test.drop('DOB', axis=1)
    categorical_features.remove('DOB')
    
    print(f"  ✓ Calculated age from DOB and ADMITTIME")
    print(f"    Age range: {X['age'].min():.1f} - {X['age'].max():.1f} years")
    print(f"    Mean age: {X['age'].mean():.1f} years")
    print(f"    Missing ages: {X['age'].isna().sum()}")
    
    # Sanity check: ages should be reasonable (0-120 years)
    if X['age'].max() > 120 or X['age'].min() < 0:
        print(f"    ⚠️ WARNING: Unusual age range detected!")
        print(f"    Sample ages: {X['age'].head(10).tolist()}")
    
    # Handle missing or invalid ages
    if X['age'].isna().sum() > 0 or (X['age'] < 0).any() or (X['age'] > 120).any():
        # Set invalid ages to NaN
        X.loc[(X['age'] < 0) | (X['age'] > 120), 'age'] = np.nan
        X_test.loc[(X_test['age'] < 0) | (X_test['age'] > 120), 'age'] = np.nan
        
        # Impute with median
        age_median = X['age'].median()
        X['age'].fillna(age_median, inplace=True)
        X_test['age'].fillna(age_median, inplace=True)
        print(f"    ✓ Imputed invalid ages with median: {age_median:.1f}")
    
    # Add to numeric features for scaling later
    if 'age' not in numeric_features:
        numeric_features.append('age')


--- Step 1: Converting DOB to age ---
  ✓ Calculated age from DOB and ADMITTIME
    Age range: 15.0 - 89.0 years
    Mean age: 62.7 years
    Missing ages: 1107
    ✓ Imputed invalid ages with median: 64.5


C:\Users\corne\AppData\Local\Temp\ipykernel_29328\1596579972.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['age'].fillna(age_median, inplace=True)
C:\Users\corne\AppData\Local\Temp\ipykernel_29328\1596579972.py:68: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [8]:
# =============================================================================
# 5.5. DIAGNOSE HIGH-CARDINALITY CATEGORICAL FEATURES
# =============================================================================
print("\n" + "="*70)
print("DIAGNOSING CATEGORICAL FEATURES")
print("="*70)

print(f"\nTotal categorical features: {len(categorical_features)}")

# Check cardinality (number of unique values) for each categorical feature
for cat_col in categorical_features:
    n_unique = X[cat_col].nunique()
    print(f"  {cat_col}: {n_unique} unique values")
    
    # Show distribution if few unique values
    if n_unique <= 10:
        print(f"    Distribution: {X[cat_col].value_counts().to_dict()}")
    else:
        print(f"    ⚠️ HIGH CARDINALITY - will create {n_unique} one-hot columns!")

# Estimate final feature count after one-hot encoding
estimated_features = len(numeric_features)
for cat_col in categorical_features:
    estimated_features += X[cat_col].nunique() - 1  # -1 because drop_first=True

print(f"\n⚠️ ESTIMATED TOTAL FEATURES AFTER ONE-HOT ENCODING: {estimated_features}")
print(f"Current numeric features: {len(numeric_features)}")


DIAGNOSING CATEGORICAL FEATURES

Total categorical features: 9
  GENDER: 2 unique values
    Distribution: {'M': 11759, 'F': 9126}
  ADMISSION_TYPE: 3 unique values
    Distribution: {'EMERGENCY': 17817, 'ELECTIVE': 2848, 'URGENT': 220}
  INSURANCE: 5 unique values
    Distribution: {'Medicare': 11718, 'Private': 6245, 'Medicaid': 2117, 'Government': 611, 'Self Pay': 194}
  RELIGION: 17 unique values
    ⚠️ HIGH CARDINALITY - will create 17 one-hot columns!
  MARITAL_STATUS: 7 unique values
    Distribution: {'MARRIED': 10386, 'SINGLE': 5910, 'WIDOWED': 2819, 'DIVORCED': 1413, 'SEPARATED': 240, 'UNKNOWN (DEFAULT)': 103, 'LIFE PARTNER': 14}
  ETHNICITY: 41 unique values
    ⚠️ HIGH CARDINALITY - will create 41 one-hot columns!
  DIAGNOSIS: 6193 unique values
    ⚠️ HIGH CARDINALITY - will create 6193 one-hot columns!
  ICD9_diagnosis: 1853 unique values
    ⚠️ HIGH CARDINALITY - will create 1853 one-hot columns!
  FIRST_CAREUNIT: 5 unique values
    Distribution: {'MICU': 8640, 'SICU':

In [9]:
# =============================================================================
# 7. SMART CATEGORICAL ENCODING
# =============================================================================
print("\n" + "="*70)
print("SMART CATEGORICAL ENCODING")
print("="*70)

# -----------------------------------------------------------------------------
# STEP 2: Handle ICD9_diagnosis (extract category, then target encode)
# -----------------------------------------------------------------------------
print("\n--- Step 2: Processing ICD9_diagnosis codes ---")

if 'ICD9_diagnosis' in X.columns:
    # ICD9 codes have hierarchical structure:
    # First digit = broad category (e.g., 4XX = circulatory system)
    # First 3 digits = more specific category
    
    def extract_icd9_category(code):
        """Extract first 3 characters from ICD9 code"""
        if pd.isna(code):
            return 'UNKNOWN'
        code_str = str(code).strip()
        # Remove decimal point and take first 3 characters
        code_str = code_str.replace('.', '')
        if len(code_str) >= 3:
            return code_str[:3]
        elif len(code_str) > 0:
            return code_str
        else:
            return 'UNKNOWN'
    
    X['ICD9_category'] = X['ICD9_diagnosis'].apply(extract_icd9_category)
    X_test['ICD9_category'] = X_test['ICD9_diagnosis'].apply(extract_icd9_category)
    
    n_icd9_categories = X['ICD9_category'].nunique()
    print(f"  ✓ Extracted ICD9 categories: {n_icd9_categories} unique categories")
    
    # Target encode (because still likely 100+ categories)
    print(f"  → Using target encoding for ICD9 categories")
    encoding_map = y.groupby(X['ICD9_category']).mean().to_dict()
    global_mean = y.mean()
    
    X['ICD9_encoded'] = X['ICD9_category'].map(encoding_map)
    X_test['ICD9_encoded'] = X_test['ICD9_category'].map(encoding_map).fillna(global_mean)
    
    # Add to numeric features (target encoding creates numeric feature)
    numeric_features.append('ICD9_encoded')
    
    # Drop originals
    X = X.drop(['ICD9_diagnosis', 'ICD9_category'], axis=1)
    X_test = X_test.drop(['ICD9_diagnosis', 'ICD9_category'], axis=1)
    categorical_features.remove('ICD9_diagnosis')
    
    print(f"  ✓ ICD9_diagnosis → ICD9_encoded (numeric)")

# -----------------------------------------------------------------------------
# STEP 3: Handle DIAGNOSIS (free text - extract keywords or drop)
# -----------------------------------------------------------------------------
print("\n--- Step 3: Processing DIAGNOSIS (free text) ---")

if 'DIAGNOSIS' in X.columns:
    # Option 1: Drop it (safest - free text is very high cardinality)
    # Option 2: Extract common keywords (more complex)
    
    # For now, let's DROP it to keep things simple
    # (We already have ICD9 codes which are more structured)
    
    print(f"  ✓ Dropping DIAGNOSIS (free text, {X['DIAGNOSIS'].nunique()} unique values)")
    print(f"    → Keeping ICD9_encoded instead (more structured)")
    
    X = X.drop('DIAGNOSIS', axis=1)
    X_test = X_test.drop('DIAGNOSIS', axis=1)
    categorical_features.remove('DIAGNOSIS')

# -----------------------------------------------------------------------------
# STEP 4: Group ETHNICITY into broader categories
# -----------------------------------------------------------------------------
print("\n--- Step 4: Grouping ETHNICITY ---")

if 'ETHNICITY' in X.columns:
    def group_ethnicity(ethnicity):
        if pd.isna(ethnicity):
            return 'UNKNOWN'
        ethnicity = str(ethnicity).upper()
        
        # WHITE (includes variants like WHITE - RUSSIAN, WHITE - BRAZILIAN, etc.)
        if 'WHITE' in ethnicity:
            return 'WHITE'
        
        # BLACK (includes BLACK/AFRICAN AMERICAN, BLACK/HAITIAN, BLACK/CAPE VERDEAN, etc.)
        elif 'BLACK' in ethnicity or 'AFRICAN' in ethnicity:
            return 'BLACK'
        
        # HISPANIC/LATINO (all variants)
        elif 'HISPANIC' in ethnicity or 'LATINO' in ethnicity:
            return 'HISPANIC'
        
        # ASIAN (includes ASIAN - CHINESE, ASIAN - VIETNAMESE, etc.)
        elif 'ASIAN' in ethnicity:
            return 'ASIAN'
        
        # NATIVE/INDIGENOUS (American Indian/Alaska Native)
        elif 'AMERICAN INDIAN' in ethnicity or 'ALASKA NATIVE' in ethnicity:
            return 'NATIVE'
        
        # PACIFIC ISLANDER
        elif 'HAWAIIAN' in ethnicity or 'PACIFIC ISLANDER' in ethnicity:
            return 'PACIFIC_ISLANDER'
        
        # UNKNOWN/NOT SPECIFIED/DECLINED
        elif any(x in ethnicity for x in ['UNKNOWN', 'UNABLE', 'DECLINED', 'NOT SPECIFIED']):
            return 'UNKNOWN'
        
        # OTHER (includes MULTI RACE, MIDDLE EASTERN, CARIBBEAN, PORTUGUESE, etc.)
        else:
            return 'OTHER'
    
    X['ETHNICITY'] = X['ETHNICITY'].apply(group_ethnicity)
    X_test['ETHNICITY'] = X_test['ETHNICITY'].apply(group_ethnicity)
    
    print(f"  ✓ Grouped ETHNICITY: 41 → {X['ETHNICITY'].nunique()} categories")
    print(f"    New categories: {sorted(X['ETHNICITY'].unique())}")
    print(f"    Distribution:")
    for cat, count in X['ETHNICITY'].value_counts().items():
        print(f"      {cat}: {count} ({count/len(X)*100:.1f}%)")

# -----------------------------------------------------------------------------
# STEP 5: Group RELIGION into broader categories
# -----------------------------------------------------------------------------
print("\n--- Step 5: Grouping RELIGION ---")

if 'RELIGION' in X.columns:
    def group_religion(religion):
        if pd.isna(religion):
            return 'UNKNOWN'
        religion = str(religion).upper()
        
        # CATHOLIC
        if 'CATHOLIC' in religion:
            return 'CATHOLIC'
        
        # PROTESTANT/CHRISTIAN (includes PROTESTANT QUAKER, EPISCOPALIAN, etc.)
        elif any(x in religion for x in ['PROTESTANT', 'EPISCOPALIAN', 'QUAKER']):
            return 'PROTESTANT'
        
        # JEWISH (includes HEBREW)
        elif 'JEWISH' in religion or 'HEBREW' in religion:
            return 'JEWISH'
        
        # MUSLIM
        elif 'MUSLIM' in religion:
            return 'MUSLIM'
        
        # ORTHODOX (GREEK ORTHODOX, ROMANIAN ORTHODOX)
        elif 'ORTHODOX' in religion:
            return 'ORTHODOX'
        
        # OTHER RELIGIONS (Buddhist, Hindu, Christian Scientist, Jehovah's Witness, etc.)
        elif any(x in religion for x in ['BUDDHIST', 'HINDU', 'JEHOVAH', 'CHRISTIAN SCIENTIST', 
                                          '7TH DAY ADVENTIST', 'UNITARIAN']):
            return 'OTHER_RELIGION'
        
        # UNKNOWN/NOT SPECIFIED
        elif any(x in religion for x in ['UNOBTAINABLE', 'NOT SPECIFIED', 'UNKNOWN']):
            return 'UNKNOWN'
        
        # OTHER
        else:
            return 'OTHER'
    
    X['RELIGION'] = X['RELIGION'].apply(group_religion)
    X_test['RELIGION'] = X_test['RELIGION'].apply(group_religion)
    
    print(f"  ✓ Grouped RELIGION: 17 → {X['RELIGION'].nunique()} categories")
    print(f"    New categories: {sorted(X['RELIGION'].unique())}")
    print(f"    Distribution:")
    for cat, count in X['RELIGION'].value_counts().items():
        print(f"      {cat}: {count} ({count/len(X)*100:.1f}%)")

# -----------------------------------------------------------------------------
# STEP 6: Group MARITAL_STATUS
# -----------------------------------------------------------------------------
print("\n--- Step 6: Grouping MARITAL_STATUS ---")

if 'MARITAL_STATUS' in X.columns:
    def group_marital_status(status):
        if pd.isna(status):
            return 'UNKNOWN'
        status = str(status).upper()
        
        # MARRIED (includes LIFE PARTNER)
        if 'MARRIED' in status or 'LIFE PARTNER' in status:
            return 'MARRIED'
        
        # SINGLE
        elif 'SINGLE' in status:
            return 'SINGLE'
        
        # WIDOWED
        elif 'WIDOWED' in status:
            return 'WIDOWED'
        
        # DIVORCED/SEPARATED (group together - both indicate ended relationship)
        elif 'DIVORCED' in status or 'SEPARATED' in status:
            return 'DIVORCED_SEPARATED'
        
        # UNKNOWN
        elif 'UNKNOWN' in status:
            return 'UNKNOWN'
        
        else:
            return 'UNKNOWN'
    
    X['MARITAL_STATUS'] = X['MARITAL_STATUS'].apply(group_marital_status)
    X_test['MARITAL_STATUS'] = X_test['MARITAL_STATUS'].apply(group_marital_status)
    
    print(f"  ✓ Grouped MARITAL_STATUS: 7 → {X['MARITAL_STATUS'].nunique()} categories")
    print(f"    New categories: {sorted(X['MARITAL_STATUS'].unique())}")
    print(f"    Distribution:")
    for cat, count in X['MARITAL_STATUS'].value_counts().items():
        print(f"      {cat}: {count} ({count/len(X)*100:.1f}%)")
# -----------------------------------------------------------------------------
# STEP 7: One-hot encode remaining low-cardinality features
# -----------------------------------------------------------------------------
print("\n--- Step 7: One-hot encoding remaining categorical features ---")

# Update categorical_features list
remaining_categorical = [col for col in categorical_features if col in X.columns]
print(f"\nFeatures to one-hot encode ({len(remaining_categorical)}):")

# Verify cardinality
total_new_features = 0
for col in remaining_categorical:
    n_unique = X[col].nunique()
    total_new_features += (n_unique - 1)  # drop_first=True
    print(f"  {col}: {n_unique} categories → {n_unique-1} binary features")

print(f"\nEstimated new binary features from one-hot encoding: {total_new_features}")

if len(remaining_categorical) > 0:
    # One-hot encode
    X_combined = pd.concat([X, X_test], keys=['train', 'test'])
    X_encoded = pd.get_dummies(X_combined, columns=remaining_categorical, drop_first=True)
    X = X_encoded.xs('train')
    X_test = X_encoded.xs('test')
    
    print(f"✓ One-hot encoding complete")

# -----------------------------------------------------------------------------
# FINAL SUMMARY
# -----------------------------------------------------------------------------
print("\n" + "="*70)
print("ENCODING COMPLETE - SUMMARY")
print("="*70)
print(f"Original numeric features: {len(numeric_features)}")
print(f"Target-encoded features: 1 (ICD9_encoded)")
print(f"Binary features from one-hot encoding: {total_new_features}")
print(f"\nFinal shapes:")
print(f"  X: {X.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  Total features: {X.shape[1]}")

# Check if reasonable
if X.shape[1] > 200:
    print(f"\n⚠️ WARNING: {X.shape[1]} features might still be too many")
    print("Consider more aggressive grouping or feature selection")
elif X.shape[1] < 50:
    print(f"\n⚠️ WARNING: Only {X.shape[1]} features - might be too few")
    print("Consider keeping more granular categories")
else:
    print(f"\n✓ Feature count looks good ({X.shape[1]} features)")

# Show a sample of the final feature names
print(f"\nSample of final features (first 20):")
print(list(X.columns[:20]))


SMART CATEGORICAL ENCODING

--- Step 2: Processing ICD9_diagnosis codes ---
  ✓ Extracted ICD9 categories: 530 unique categories
  → Using target encoding for ICD9 categories
  ✓ ICD9_diagnosis → ICD9_encoded (numeric)

--- Step 3: Processing DIAGNOSIS (free text) ---
  ✓ Dropping DIAGNOSIS (free text, 6193 unique values)
    → Keeping ICD9_encoded instead (more structured)

--- Step 4: Grouping ETHNICITY ---
  ✓ Grouped ETHNICITY: 41 → 8 categories
    New categories: ['ASIAN', 'BLACK', 'HISPANIC', 'NATIVE', 'OTHER', 'PACIFIC_ISLANDER', 'UNKNOWN', 'WHITE']
    Distribution:
      WHITE: 15330 (73.4%)
      BLACK: 2201 (10.5%)
      UNKNOWN: 1320 (6.3%)
      HISPANIC: 852 (4.1%)
      OTHER: 616 (2.9%)
      ASIAN: 545 (2.6%)
      NATIVE: 15 (0.1%)
      PACIFIC_ISLANDER: 6 (0.0%)

--- Step 5: Grouping RELIGION ---
  ✓ Grouped RELIGION: 17 → 8 categories
    New categories: ['CATHOLIC', 'JEWISH', 'MUSLIM', 'ORTHODOX', 'OTHER', 'OTHER_RELIGION', 'PROTESTANT', 'UNKNOWN']
    Distribut

In [10]:
# =============================================================================
# 7.5. TRANSFORM SKEWED FEATURES
# =============================================================================
print("\n" + "="*70)
print("TRANSFORMING SKEWED FEATURES")
print("="*70)

# Apply log transformation to highly skewed features
# This helps models learn better from skewed distributions

skewed_features_to_transform = [
    'Glucose_Max', 'Glucose_Mean', 'Glucose_Min', 'Glucose_Range',
    'MeanBP_Max', 'Temp_Range', 'age_squared'
]

for feat in skewed_features_to_transform:
    if feat in X.columns:
        # Add 1 to handle any zeros, then log transform
        X[feat] = np.log1p(X[feat] - X[feat].min() + 1)
        X_test[feat] = np.log1p(X_test[feat] - X_test[feat].min() + 1)
        print(f"  ✓ Log-transformed {feat}")

print(f"\nTransformed {len([f for f in skewed_features_to_transform if f in X.columns])} skewed features")


TRANSFORMING SKEWED FEATURES
  ✓ Log-transformed Glucose_Max
  ✓ Log-transformed Glucose_Mean
  ✓ Log-transformed Glucose_Min
  ✓ Log-transformed MeanBP_Max

Transformed 4 skewed features


C:\Users\corne\AppData\Local\Temp\ipykernel_29328\953503949.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feat] = np.log1p(X[feat] - X[feat].min() + 1)
C:\Users\corne\AppData\Local\Temp\ipykernel_29328\953503949.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feat] = np.log1p(X_test[feat] - X_test[feat].min() + 1)


In [11]:
# =============================================================================
# 8. FEATURE SCALING
# =============================================================================
print("\n" + "="*70)
print("SCALING NUMERIC FEATURES")
print("="*70)

# Identify numeric columns after encoding (one-hot creates binary columns)
# Only scale original numeric features + engineered features
numeric_cols_to_scale = [col for col in numeric_features if col in X.columns]

# Add engineered features to the list (they were created after numeric_features was defined)
engineered_numeric = ['PulsePressure', 'SysBP_Range', 'ShockIndex', 'ModifiedShockIndex',
                      'RespDistress_Score', 'Temp_Range', 'Glucose_Range', 'HeartRate_Range',
                      'age_squared', 'Severity_Score']
for feat in engineered_numeric:
    if feat in X.columns and feat not in numeric_cols_to_scale:
        numeric_cols_to_scale.append(feat)

print(f"Total numeric features to process: {len(numeric_cols_to_scale)}")

# Separate binary indicators from continuous features
# Binary features should NOT be scaled (would destroy their meaning)
binary_features = []
continuous_features = []

for col in numeric_cols_to_scale:
    unique_count = X[col].nunique()
    unique_values = set(X[col].unique())
    
    # Check if binary (only 0 and 1, or just one value after one-hot encoding)
    is_binary = (unique_count <= 2 and 
                 (unique_values.issubset({0, 1}) or 
                  unique_values.issubset({0.0, 1.0}) or
                  unique_values.issubset({0, 1, 0.0, 1.0})))
    
    if is_binary:
        binary_features.append(col)
    else:
        continuous_features.append(col)

print(f"\nFeature classification:")
print(f"  Binary features (will NOT scale): {len(binary_features)}")
print(f"  Continuous features (will scale): {len(continuous_features)}")

# Scale only continuous features
scaler = StandardScaler()

if len(continuous_features) > 0:
    X[continuous_features] = scaler.fit_transform(X[continuous_features])
    X_test[continuous_features] = scaler.transform(X_test[continuous_features])
    print(f"\n✓ Scaled {len(continuous_features)} continuous features")
else:
    print("\n⚠ No continuous features found to scale")

print(f"✓ Left {len(binary_features)} binary features unscaled")

# Verify some binary features retained their 0/1 values
if len(binary_features) > 0:
    print("\nVerifying binary features (sample of 5):")
    for feat in binary_features[:5]:
        unique_vals = sorted(X[feat].unique())
        print(f"  {feat}: {unique_vals}")


SCALING NUMERIC FEATURES
Total numeric features to process: 26

Feature classification:
  Binary features (will NOT scale): 0
  Continuous features (will scale): 26

✓ Scaled 26 continuous features
✓ Left 0 binary features unscaled


C:\Users\corne\AppData\Local\Temp\ipykernel_29328\2512229183.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[continuous_features] = scaler.fit_transform(X[continuous_features])
C:\Users\corne\AppData\Local\Temp\ipykernel_29328\2512229183.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[continuous_features] = scaler.transform(X_test[continuous_features])


In [12]:
# =============================================================================
# 8.5. FEATURE ENGINEERING - MEDICAL DOMAIN KNOWLEDGE
# =============================================================================
print("\n" + "="*70)
print("FEATURE ENGINEERING")
print("="*70)

original_feature_count = X.shape[1]

# -----------------------------------------------------------------------------
# Blood Pressure Features
# -----------------------------------------------------------------------------
print("\n--- Blood Pressure Features ---")

# Pulse Pressure (SysBP - DiasBP) - cardiovascular health indicator
if all(col in X.columns for col in ['SysBP_Mean', 'DiasBP_Mean']):
    X['PulsePressure'] = X['SysBP_Mean'] - X['DiasBP_Mean']
    X_test['PulsePressure'] = X_test['SysBP_Mean'] - X_test['DiasBP_Mean']
    print("  ✓ Pulse pressure")

# Blood pressure variability
if all(col in X.columns for col in ['SysBP_Min', 'SysBP_Max']):
    X['SysBP_Range'] = X['SysBP_Max'] - X['SysBP_Min']
    X_test['SysBP_Range'] = X_test['SysBP_Max'] - X_test['SysBP_Min']
    print("  ✓ Systolic BP range")

# -----------------------------------------------------------------------------
# Shock Indices (Critical for ICU mortality prediction)
# -----------------------------------------------------------------------------
print("\n--- Shock Indices ---")

# Shock Index = HR / SysBP (>0.9 indicates shock)
if all(col in X.columns for col in ['HeartRate_Mean', 'SysBP_Mean']):
    X['ShockIndex'] = X['HeartRate_Mean'] / (X['SysBP_Mean'] + 1)
    X_test['ShockIndex'] = X_test['HeartRate_Mean'] / (X_test['SysBP_Mean'] + 1)
    
    # Cap extreme values (clinical range: 0.5-2.0 is reasonable)
    X['ShockIndex'] = X['ShockIndex'].clip(0, 3)
    X_test['ShockIndex'] = X_test['ShockIndex'].clip(0, 3)
    print("  ✓ Shock index (HR/SysBP) - capped at [0, 3]")

# Modified Shock Index = HR / MAP
if all(col in X.columns for col in ['HeartRate_Mean', 'MeanBP_Mean']):
    X['ModifiedShockIndex'] = X['HeartRate_Mean'] / (X['MeanBP_Mean'] + 1)
    X_test['ModifiedShockIndex'] = X_test['HeartRate_Mean'] / (X_test['MeanBP_Mean'] + 1)
    
    # Cap extreme values
    X['ModifiedShockIndex'] = X['ModifiedShockIndex'].clip(0, 3)
    X_test['ModifiedShockIndex'] = X_test['ModifiedShockIndex'].clip(0, 3)
    print("  ✓ Modified shock index (HR/MAP) - capped at [0, 3]")
    
# -----------------------------------------------------------------------------
# Respiratory Features
# -----------------------------------------------------------------------------
print("\n--- Respiratory Features ---")

# Hypoxemia indicator (SpO2 < 90% is clinically significant)
if 'SpO2_Min' in X.columns:
    X['Hypoxemia'] = (X['SpO2_Min'] < 90).astype(int)
    X_test['Hypoxemia'] = (X_test['SpO2_Min'] < 90).astype(int)
    print("  ✓ Hypoxemia (SpO2 < 90%)")

# Respiratory rate abnormality (normal: 12-20 breaths/min)
if 'RespRate_Mean' in X.columns:
    X['RespRate_Abnormal'] = ((X['RespRate_Mean'] < 12) | (X['RespRate_Mean'] > 20)).astype(int)
    X_test['RespRate_Abnormal'] = ((X_test['RespRate_Mean'] < 12) | (X_test['RespRate_Mean'] > 20)).astype(int)
    print("  ✓ Abnormal respiratory rate")

# Respiratory distress score (high RR + low SpO2)
if all(col in X.columns for col in ['RespRate_Mean', 'SpO2_Mean']):
    X['RespDistress_Score'] = X['RespRate_Mean'] * (100 - X['SpO2_Mean'])
    X_test['RespDistress_Score'] = X_test['RespRate_Mean'] * (100 - X_test['SpO2_Mean'])
    print("  ✓ Respiratory distress score")

# -----------------------------------------------------------------------------
# Temperature Features
# -----------------------------------------------------------------------------
print("\n--- Temperature Features ---")

# Fever (>38°C)
if 'TempC_Max' in X.columns:
    X['Fever'] = (X['TempC_Max'] > 38).astype(int)
    X_test['Fever'] = (X_test['TempC_Max'] > 38).astype(int)
    print("  ✓ Fever indicator")

# Hypothermia (<36°C)
if 'TempC_Min' in X.columns:
    X['Hypothermia'] = (X['TempC_Min'] < 36).astype(int)
    X_test['Hypothermia'] = (X_test['TempC_Min'] < 36).astype(int)
    print("  ✓ Hypothermia indicator")

# Temperature instability
if all(col in X.columns for col in ['TempC_Min', 'TempC_Max']):
    X['Temp_Range'] = X['TempC_Max'] - X['TempC_Min']
    X_test['Temp_Range'] = X_test['TempC_Max'] - X_test['TempC_Min']
    print("  ✓ Temperature range")

# -----------------------------------------------------------------------------
# Glucose Features
# -----------------------------------------------------------------------------
print("\n--- Glucose Features ---")

# Hyperglycemia (>180 mg/dL)
if 'Glucose_Max' in X.columns:
    X['Hyperglycemia'] = (X['Glucose_Max'] > 180).astype(int)
    X_test['Hyperglycemia'] = (X_test['Glucose_Max'] > 180).astype(int)
    print("  ✓ Hyperglycemia (>180 mg/dL)")

# Hypoglycemia (<70 mg/dL)
if 'Glucose_Min' in X.columns:
    X['Hypoglycemia'] = (X['Glucose_Min'] < 70).astype(int)
    X_test['Hypoglycemia'] = (X_test['Glucose_Min'] < 70).astype(int)
    print("  ✓ Hypoglycemia (<70 mg/dL)")

# Glucose variability
if all(col in X.columns for col in ['Glucose_Min', 'Glucose_Max']):
    X['Glucose_Range'] = X['Glucose_Max'] - X['Glucose_Min']
    X_test['Glucose_Range'] = X_test['Glucose_Max'] - X_test['Glucose_Min']
    print("  ✓ Glucose variability")

# -----------------------------------------------------------------------------
# Age-Related Features
# -----------------------------------------------------------------------------
print("\n--- Age-Related Features ---")

# Elderly indicator (>65 years)
if 'age' in X.columns:
    X['Elderly'] = (X['age'] > 65).astype(int)
    X_test['Elderly'] = (X_test['age'] > 65).astype(int)
    print("  ✓ Elderly indicator (>65 years)")
    
    # Age squared (capture non-linear effects)
    X['age_squared'] = X['age'] ** 2
    X_test['age_squared'] = X_test['age'] ** 2
    print("  ✓ Age squared")

# -----------------------------------------------------------------------------
# Vital Sign Variability
# -----------------------------------------------------------------------------
print("\n--- Vital Sign Variability ---")

# Heart rate variability
if all(col in X.columns for col in ['HeartRate_Min', 'HeartRate_Max']):
    X['HeartRate_Range'] = X['HeartRate_Max'] - X['HeartRate_Min']
    X_test['HeartRate_Range'] = X_test['HeartRate_Max'] - X_test['HeartRate_Min']
    print("  ✓ Heart rate range")

# -----------------------------------------------------------------------------
# Composite Risk Score
# -----------------------------------------------------------------------------
print("\n--- Composite Risk Score ---")

# Count abnormal vital signs
severity_components = []

if 'ShockIndex' in X.columns:
    severity_components.append((X['ShockIndex'] > 0.9).astype(int))
if 'Hypoxemia' in X.columns:
    severity_components.append(X['Hypoxemia'])
if 'RespRate_Abnormal' in X.columns:
    severity_components.append(X['RespRate_Abnormal'])
if 'Fever' in X.columns:
    severity_components.append(X['Fever'])
if 'Hypothermia' in X.columns:
    severity_components.append(X['Hypothermia'])

if severity_components:
    X['Severity_Score'] = sum(severity_components)
    
    # Repeat for test set
    severity_components_test = []
    if 'ShockIndex' in X_test.columns:
        severity_components_test.append((X_test['ShockIndex'] > 0.9).astype(int))
    if 'Hypoxemia' in X_test.columns:
        severity_components_test.append(X_test['Hypoxemia'])
    if 'RespRate_Abnormal' in X_test.columns:
        severity_components_test.append(X_test['RespRate_Abnormal'])
    if 'Fever' in X_test.columns:
        severity_components_test.append(X_test['Fever'])
    if 'Hypothermia' in X_test.columns:
        severity_components_test.append(X_test['Hypothermia'])
    
    X_test['Severity_Score'] = sum(severity_components_test)
    print("  ✓ Composite severity score (0-5)")

# -----------------------------------------------------------------------------
# Summary
# -----------------------------------------------------------------------------
new_feature_count = X.shape[1]
added_features = new_feature_count - original_feature_count

print(f"\n{'='*70}")
print("FEATURE ENGINEERING COMPLETE")
print(f"{'='*70}")
print(f"Original features: {original_feature_count}")
print(f"New features: {new_feature_count}")
print(f"Added: {added_features} engineered features")

# Update numeric_features list to include new engineered features
new_engineered_features = [col for col in X.columns if col not in numeric_features + ['ICD9_encoded']]
numeric_features.extend(new_engineered_features)

print(f"\nEngineered features added to numeric_features list for scaling")

# -----------------------------------------------------------------------------
# Remove Redundant Features
# -----------------------------------------------------------------------------
print("\n--- Removing Redundant Features ---")

redundant_features = []

# RespDistress_Score is 99.99% correlated with RespRate_Mean
# Keep RespRate_Mean (original feature) and drop engineered one
if 'RespDistress_Score' in X.columns:
    X = X.drop('RespDistress_Score', axis=1)
    X_test = X_test.drop('RespDistress_Score', axis=1)
    redundant_features.append('RespDistress_Score')
    print("  ✓ Dropped RespDistress_Score (redundant with RespRate_Mean)")

# MeanBP_Mean is 90% correlated with DiasBP_Mean
# Keep DiasBP_Mean and drop MeanBP_Mean
if 'MeanBP_Mean' in X.columns:
    X = X.drop('MeanBP_Mean', axis=1)
    X_test = X_test.drop('MeanBP_Mean', axis=1)
    redundant_features.append('MeanBP_Mean')
    print("  ✓ Dropped MeanBP_Mean (redundant with DiasBP_Mean)")

if redundant_features:
    print(f"\nRemoved {len(redundant_features)} redundant features")



FEATURE ENGINEERING

--- Blood Pressure Features ---
  ✓ Pulse pressure
  ✓ Systolic BP range

--- Shock Indices ---
  ✓ Shock index (HR/SysBP) - capped at [0, 3]
  ✓ Modified shock index (HR/MAP) - capped at [0, 3]

--- Respiratory Features ---
  ✓ Hypoxemia (SpO2 < 90%)
  ✓ Abnormal respiratory rate
  ✓ Respiratory distress score

--- Temperature Features ---
  ✓ Fever indicator
  ✓ Hypothermia indicator
  ✓ Temperature range

--- Glucose Features ---
  ✓ Hyperglycemia (>180 mg/dL)
  ✓ Hypoglycemia (<70 mg/dL)
  ✓ Glucose variability

--- Age-Related Features ---
  ✓ Elderly indicator (>65 years)
  ✓ Age squared

--- Vital Sign Variability ---
  ✓ Heart rate range

--- Composite Risk Score ---
  ✓ Composite severity score (0-5)

FEATURE ENGINEERING COMPLETE
Original features: 55
New features: 72
Added: 17 engineered features

Engineered features added to numeric_features list for scaling

--- Removing Redundant Features ---
  ✓ Dropped RespDistress_Score (redundant with RespRate_Mea

C:\Users\corne\AppData\Local\Temp\ipykernel_29328\2847557397.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['PulsePressure'] = X['SysBP_Mean'] - X['DiasBP_Mean']
C:\Users\corne\AppData\Local\Temp\ipykernel_29328\2847557397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['PulsePressure'] = X_test['SysBP_Mean'] - X_test['DiasBP_Mean']
C:\Users\corne\AppData\Local\Temp\ipykernel_29328\2847557397.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [13]:
# =============================================================================
# 8. SAVE PROCESSED DATA
# =============================================================================
print("\n--- Saving processed data ---")

import os
os.makedirs('../data/processed', exist_ok=True)

# Save as pickle (preserves dtypes and column names)
X.to_pickle('../data/processed/X_train_processed.pkl')
y.to_pickle('../data/processed/y_train.pkl')
X_test.to_pickle('../data/processed/X_test_processed.pkl')
test_ids.to_pickle('../data/processed/test_ids.pkl')

# Also save preprocessing objects (to use on new data if needed)
with open('../data/processed/numeric_imputer.pkl', 'wb') as f:
    pickle.dump(numeric_imputer, f)
with open('../data/processed/categorical_imputer.pkl', 'wb') as f:
    pickle.dump(categorical_imputer, f)
with open('../data/processed/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("✓ Processed data saved to ../data/processed/")
print("\n" + "="*70)
print("PREPROCESSING COMPLETE!")
print("="*70)
print("\nYou can now run modeling notebooks without repeating preprocessing.")


--- Saving processed data ---
✓ Processed data saved to ../data/processed/

PREPROCESSING COMPLETE!

You can now run modeling notebooks without repeating preprocessing.
